In [17]:
from mandala.imports import *

# the storage saves calls and tracks dependencies, versions, etc.
storage = Storage( 
    deps_path='__main__' # track dependencies in current session
    ) 

@op # memoization (and more) decorator
def increment(x: int) -> int: # always indicate number of outputs in return type
  print('hi from increment!')
  return x + 1

increment(23) # function acts normally

with storage.run(): # context manager that triggers `mandala`
  y = increment(23) # now it's memoized w.r.t. this version of `increment`

print(y) # result wrapped with metadata. 
print(unwrap(y)) # `unwrap` gets the raw value

with storage.run():
  y = increment(23) # loads result from `storage`; doesn't execute `increment`

@op # type-annotate data structures to store elts separately
def average(nums: list) -> float: 
  print('hi from average!')
  return sum(nums) / len(nums)

# memoized functions are designed to be composed!
with storage.run(): 
    # sliding averages of `increment`'s results over 3 elts
    nums = [increment(i) for i in range(5)]
    for i in range(3):
        result = average(nums[i:i+3])

# get a table of all values similar to `result` in `storage`,
# i.e., computed as average([increment(something), ...])
# read the message this prints out!
print(storage.similar(result, context=True))

# change implementation of `increment` and re-run
# you'll be asked if the change requires recomputing dependencies (say yes)
@op
def increment(x: int) -> int:
  print('hi from new increment!')
  return x + 2

with storage.run(): 
    nums = [increment(i) for i in range(5)]
    for i in range(3):
        # only one call to `average` is executed!
        result = average(nums[i:i+3])

# query is ran against the *new* version of `increment`
print(storage.similar(result, context=True))

hi from increment!
hi from increment!
ValueRef(24, uid=f8b...)
24
hi from increment!
hi from increment!
hi from increment!
hi from increment!
hi from increment!
hi from average!
hi from average!
hi from average!
Pattern-matching to the following computational graph (all constraints apply):
    idx0 = Q() # index into list
    a0 = Q() # input to computation; can match anything
    a1 = increment(x=a0)
    a2 = ListQ(elts=[a1], idxs=[idx0]) # a2 will match any list containing a match for a1 at index idx0
    result = average(nums=a2)
    result = storage.df(idx0, a0, a1, a2, result)
   idx0  a0  a1         a2  result
8     0   0   1  [1, 2, 3]     2.0
6     1   1   2  [1, 2, 3]     2.0
2     2   2   3  [1, 2, 3]     2.0
7     0   1   2  [2, 3, 4]     3.0
3     1   2   3  [2, 3, 4]     3.0
5     2   3   4  [2, 3, 4]     3.0
1     0   2   3  [3, 4, 5]     4.0
4     1   3   4  [3, 4, 5]     4.0
0     2   4   5  [3, 4, 5]     4.0
CHANGE DETECTED in increment from module __main__
Dependent c